# Manutenzione database

- [Dimensione database](#dimensione-file)
- [Conteggio righe tabelle](#conteggio-righe)
- [Frammentazione indici](#frammentazione-indici)
- [Rebuild indici](#rebuild-indici)
- [Connessioni con query attive](#query-attive)

## Dimensione database

In [ ]:
SELECT
  DB_NAME() AS DbName,
  name AS FileName,
  type_desc,
  size / 128.0 AS CurrentSizeMB,
  size / 128.0 - CAST(FILEPROPERTY(name, 'SpaceUsed') AS INT) / 128.0 AS FreeSpaceMB
FROM
  sys.database_files
WHERE
  type IN (0, 1);

## Conteggio righe

In [ ]:
SELECT
  t.NAME AS TableName,
  i.name AS indexName,
  p.[Rows],
  sum(a.total_pages) AS TotalPages,
  sum(a.used_pages) AS UsedPages,
  sum(a.data_pages) AS DataPages,
  (sum(a.total_pages) * 8) / 1024 AS TotalSpaceMB,
  (sum(a.used_pages) * 8) / 1024 AS UsedSpaceMB,
  (sum(a.data_pages) * 8) / 1024 AS DataSpaceMB
FROM
  sys.tables t
  INNER JOIN sys.indexes i ON t.OBJECT_ID = i.object_id
  INNER JOIN sys.partitions p ON i.object_id = p.OBJECT_ID
  AND i.index_id = p.index_id
  INNER JOIN sys.allocation_units a ON p.partition_id = a.container_id
WHERE
  t.NAME NOT LIKE 'dt%'
  AND i.OBJECT_ID > 255
  AND i.index_id <= 1
GROUP BY
  t.NAME,
  i.object_id,
  i.index_id,
  i.name,
  p.[Rows]
ORDER BY
  object_name(i.object_id)

## Frammentazione indici

In [ ]:
SELECT
  S.name AS 'Schema',
  T.name AS 'Table',
  I.name AS 'Index',
  DDIPS.avg_fragmentation_in_percent,
  DDIPS.page_count
FROM
  sys.dm_db_index_physical_stats (DB_ID(), NULL, NULL, NULL, NULL) AS DDIPS
  INNER JOIN sys.tables T ON T.object_id = DDIPS.object_id
  INNER JOIN sys.schemas S ON T.schema_id = S.schema_id
  INNER JOIN sys.indexes I ON I.object_id = DDIPS.object_id
  AND DDIPS.index_id = I.index_id
WHERE
  DDIPS.database_id = DB_ID()
  AND I.name IS NOT NULL
  AND DDIPS.avg_fragmentation_in_percent > 0
ORDER BY
  DDIPS.avg_fragmentation_in_percent DESC

## Rebuild indici

In [ ]:
-- IMPOSTATE IL NOME DEL DB
DECLARE @Database NVARCHAR(255)   = ''

DECLARE @Table NVARCHAR(255)  
DECLARE @cmd NVARCHAR(1000)

SET @cmd = 'DECLARE TableCursor CURSOR READ_ONLY FOR SELECT ''['' + table_catalog + ''].['' + table_schema + ''].['' +  
   table_name + '']'' as tableName FROM [' + @Database + '].INFORMATION_SCHEMA.TABLES WHERE table_type = ''BASE TABLE'''   

   -- create table cursor  
   EXEC (@cmd)  
   OPEN TableCursor   

   FETCH NEXT FROM TableCursor INTO @Table   
   WHILE @@FETCH_STATUS = 0   
   BEGIN
      BEGIN TRY   
         SET @cmd = 'ALTER INDEX ALL ON ' + @Table + ' REBUILD'
         --PRINT @cmd -- uncomment if you want to see commands
         EXEC (@cmd)
      END TRY
      BEGIN CATCH
         PRINT '---'
         PRINT @cmd
         PRINT ERROR_MESSAGE()
         PRINT '---'
      END CATCH

      FETCH NEXT FROM TableCursor INTO @Table   
   END   

   CLOSE TableCursor   
   DEALLOCATE TableCursor 


## Query attive

In [ ]:
-- IMPOSTATE IL NOME DEL DB
DECLARE @databasename varchar(100) = ''

SELECT
  spid,
  hostname,
  program_name,
  cpu,
  physical_io,
  memusage,
  login_time,
  last_batch,
  status,
  cmd,
  (
    SELECT
      text
    FROM
      fn_get_sql(sql_handle)
  )
FROM
  master..sysprocesses
WHERE
  dbid = db_id(@databasename)
ORDER BY
  cpu DESC